# Pegasus Fine-tuning and Inference in Google Colab

This notebook fine-tunes Pegasus model on your dataset and runs inference pipeline.

## Setup
1. Upload your data to Google Drive
2. Update the `DRIVE_DATA_PATH` variable below
3. Run all cells


In [ ]:
MODEL = 'google/pegasus-large'
# MODEL = 'google/pegasus-cnn_dailymail'


BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 10
OUT_DIR = 'results-pegasus/2k_samples'
MAX_LENGTH = 1024 # Maximum context length to consider while preparing dataset.
epoch_metrics = []
DRIVE_DATA_PATH = "/content/drive/MyDrive/processed/10k_samples"   # UPDATE PATH
CLEAN_TEXT_COLUMN='article'
SUMMARY_COLUMN='highlights'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate
!pip install rouge_score
!pip install tqdm

In [ ]:
import torch
import pprint
import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

from transformers import (
    PegasusForConditionalGeneration,  Trainer, TrainingArguments,
    PegasusTokenizer,EarlyStoppingCallback,T5ForConditionalGeneration, T5Tokenizer,
    PegasusXForConditionalGeneration,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
)

from datasets import load_dataset

pp = pprint.PrettyPrinter()


In [ ]:

def prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  """
  Prepare configurations and base model for fine-tuning
  """
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  if freeze_encoder:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

  if val_dataset is not None:
    training_args = Seq2SeqTrainingArguments(
      output_dir=OUT_DIR,           # output directory
      num_train_epochs=10,           # total number of training epochs
      per_device_train_batch_size=18,   # batch size per device during training, can increase if memory allows
      per_device_eval_batch_size=16,    # batch size for evaluation, can increase if memory allows
      save_steps=100,                  # number of updates steps before checkpoint saves
      save_total_limit=10,              # limit the total amount of checkpoints and deletes the older checkpoints
      eval_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
      warmup_steps=100,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
      learning_rate=0.00003,
      predict_with_generate=True,
          bf16=True,
    fp16=False,
    tf32=True,

    torch_compile=True,
      report_to='tensorboard',
          metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True
    )

    trainer = Seq2SeqTrainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,            # evaluation dataset
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

  else:
    training_args = Seq2SeqTrainingArguments(
      output_dir=OUT_DIR,           # output directory
      num_train_epochs=10,           # total number of training epochs
      per_device_train_batch_size=8,   # batch size per device during training, can increase if memory allows
      save_steps=100,                  # number of updates steps before checkpoint saves
      save_total_limit=5,              # limit the total amount of checkpoints and deletes the older checkpoints
      warmup_steps=100,                # number of warmup steps for learning rate scheduler
      weight_decay=0.01,               # strength of weight decay
      logging_dir='./logs',            # directory for storing logs
      logging_steps=100,
      learning_rate=0.00003,
      predict_with_generate=True,
      report_to='tensorboard',
      eval_strategy='steps',     # evaluation strategy to adopt during training
      eval_steps=100,                  # number of update steps before evaluation
    bf16=True,
    fp16=False,
    tf32=True,

    torch_compile=True,
          metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end=True
    )

    trainer = Seq2SeqTrainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      tokenizer=tokenizer,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

  return trainer

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from transformers import TrainerCallback
import time
writer = SummaryWriter(log_dir=OUT_DIR)

class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels['input_ids'])  # len(self.labels)


    def on_step_end(self, args, state, control, **kwargs):
        if torch.cuda.is_available():
            gpu_mem = torch.cuda.memory_allocated() / (1024 ** 3)
        return control

class GpuLoggerCallback(TrainerCallback):
    def __init__(self, writer):
        self.writer = writer

    def on_step_end(self, args, state, control, **kwargs):
        if torch.cuda.is_available():
            gpu_mem = torch.cuda.memory_allocated() / (1024 ** 3)
            self.writer.add_scalar("gpu_memory_gb", gpu_mem, state.global_step)
        return control


def prepare_data(model_name,
                 train_texts, train_labels,
                 val_texts=None, val_labels=None,
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = PegasusTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True)
    decodings = tokenizer(labels, truncation=True, padding=True)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset, tokenizer

def compute_metrics(eval_pred):
    rouge = evaluate.load("rouge")
    # predictions, labels = eval_pred.predictions[0], eval_pred.label_ids[0]
    predictions, labels = eval_pred.predictions, eval_pred.label_ids


    # Decode the predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in labels before decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE
    rouge_result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"]
    )
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    rouge_result["gen_len"] = np.mean(prediction_lens)

    # NEW: Log to TensorBoard
    for k, v in rouge_result.items():
        writer.add_scalar(f"eval/{k}", v, trainer.state.global_step)
    # Print rounded values
    # pprint.print({k: round(v, 4) for k, v in rouge_result.items()})

    # Must return a *dict*, not a set
    return {k: v for k, v in rouge_result.items()}

## Fine-tune Pegasus Model


In [ ]:
from datasets import Dataset
import pandas as pd
  # train_texts, train_labels = dataset['train']['document'][:1000], dataset['train']['summary'][:1000]

print("Loading data...")
train_df = pd.read_csv(f"{DRIVE_DATA_PATH}/train.csv").head(2000)
val_df = pd.read_csv(f"{DRIVE_DATA_PATH}/val.csv").head(400)


dataset_train = Dataset.from_pandas(train_df)
dataset_valid = Dataset.from_pandas(val_df)
train_texts, train_labels = dataset_train[CLEAN_TEXT_COLUMN][:2000], dataset_train[SUMMARY_COLUMN][:2000]
valid_texts, valid_labels = dataset_valid[CLEAN_TEXT_COLUMN][:2000], dataset_valid[SUMMARY_COLUMN][:2000]



# train_texts = train_df[CLEAN_TEXT_COLUMN]
# train_labels = train_df[SUMMARY_COLUMN]
print("Train:", len(train_texts))
print("Val:", len(valid_texts))


train_dataset, val_dataset, test_dataset, tokenizer = prepare_data(MODEL, train_texts, train_labels, valid_texts, valid_labels)
trainer = prepare_fine_tuning(MODEL, tokenizer, train_dataset, val_dataset=val_dataset)
trainer.train()
trainer.add_callback(GpuLoggerCallback(writer))
start = time.time()
history = trainer.train()
end = time.time()


# dataset_train = Dataset.from_pandas(train_df)
# dataset_valid = Dataset.from_pandas(val_df)

In [ ]:
from transformers.trainer_utils import get_last_checkpoint
path = f'results'
last_ckpt = get_last_checkpoint(path)
model_path = last_ckpt if last_ckpt else OUT_DIR


model = PegasusForConditionalGeneration.from_pretrained(model_path)
tokenizer = PegasusForConditionalGeneration.from_pretrained(model_path)

# model = T5ForConditionalGeneration.from_pretrained

## Run Inference Pipeline


In [ ]:
def summarize_text(text, model, tokenizer, max_length=512, num_beams=5):
    # 1. Tokenize properly (returns attention mask too)
    encoded = tokenizer(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True,
    )

    # 2. Move everything to the same device as the model
    device = model.device
    encoded = {k: v.to(device) for k, v in encoded.items()}

    # 3. Generate summary
    summary_ids = model.generate(
        **encoded,
        max_length=128,         # not 50 → 50 is too short for news
        num_beams=num_beams,
        length_penalty=1.1,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    # 4. Decode
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nli_tok = AutoTokenizer.from_pretrained("roberta-large-mnli")
nli_model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli").to(device)

def hallucination_rate(summary, source):
    sentences = nltk.sent_tokenize(summary)
    hallucinated = 0

    for sent in sentences:
        inputs = nli_tok.encode_plus(source, sent, return_tensors="pt", truncation=True).to(device)
        logits = nli_model(**inputs).logits
        probs = torch.softmax(logits, dim=1)[0]
        contradiction = probs[0].item()
        entailment = probs[2].item()

        if contradiction > entailment:
            hallucinated += 1

    return hallucinated / len(sentences)


In [ ]:
test_df = pd.read_csv(f"{DRIVE_DATA_PATH}/test.csv")


In [ ]:
!rm -rf '/content/results-pegasus/2k_samples/checkpoint-1100'
# !rm -rf '/content/results_t5_large_regularized'

In [ ]:
from google.colab import auth
auth.authenticate_user()

# Install gcsfuse
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# Create a local directory for mounting
# !mkdir results_t5base

# Mount the GCS bucket
# Replace 'your-bucket-name' with the actual name of your GCS bucket
# !gcsfuse --implicit-dirs models_checkpoint results_t5base

!gsutil -m cp -r /content/results-pegasus/ gs://pegasus_2k_model/models/results_pegasus-large/results

